In [76]:
import glob
import os
import time
from shutil import copyfile
import json
import argparse
import cv2

import numpy as np
from PIL import Image

import matplotlib.pyplot


with open('coco_example.json') as json_file:
    data = json.load(json_file)
 
    # Print the type of data variable
    print("Type:", type(data))

# Print the data of dictionary
print(data)


Type: <class 'dict'>
{'info': {'description': 'Pole Dataset', 'url': 'https://geranos.ethz.ch/', 'version': '1.0', 'year': 2022, 'contributor': 'Geranos ETHZ', 'date_created': '2022/04/27'}, 'licenses': [{'url': 'Unknown', 'id': 1, 'name': 'NONE'}], 'images': [{'license': 1, 'file_name': 'img1.png', 'height': 4032, 'width': 3024, 'date_captured': '2022-04-26 11:37:19', 'coco_url': 'Unknown', 'flickr_url': 'Unknown', 'id': 17}], 'annotations': [{'segmentation': [], 'num_keypoints': 8, 'area': 5463.6864, 'iscrowd': 0, 'keypoints': [0, 0, 0, 35.28241517030532, 14.49119719001639, 2, 58.50563228959729, 15.153431988131127, 2, 0, 0, 0, 0, 0, 0, 47.64013378960977, 96.13742100548514, 2, 56.149545654611465, 97.43861975465072, 2, 0, 0, 0], 'image_id': 17, 'bbox': [204.01, 235.08, 60.84, 177.36], 'category_id': 420, 'id': 100000}], 'categories': [{'supercategory': 'poles', 'id': 420, 'name': 'pole', 'keypoints': ['star', 'rhomb', 'torp', 'tear', 'arrow', 'triangle', 'ninja', 'bolt'], 'skeleton': [

In [77]:
annotations_new = data.get('annotations')
keypoints_old = annotations_new[0].get('keypoints')
data_new = data
keypoints_new = []
m = []
pixel_x = []
pixel_y = []
width = 3024
height = 4032

for i in range(int(len(keypoints_old)/3)):
    pixel_x.append(keypoints_old[3*i]*width/100)
    pixel_y.append(keypoints_old[3*i+1]*height/100)
    m.append(keypoints_old[3*i+2])

for i in range(int(len(keypoints_old)/3)):
    
    keypoints_new.append(pixel_x[i])
    keypoints_new.append(pixel_y[i])
    keypoints_new.append(m[i])
    
max_pixel_x = max(pixel_x)
max_pixel_y = max(pixel_y)
    
for i in range(int(len(keypoints_old)/3)):
    if pixel_x[i] < 0.001: 
        pixel_x[i]+=10000
    if pixel_y[i] < 0.001: 
        pixel_y[i]+=10000

In [78]:
annotations_new[0]['keypoints'] = keypoints_new
bbox_new = [min(pixel_x),min(pixel_y),max_pixel_x-min(pixel_x),max_pixel_y-min(pixel_y)]
annotations_new[0]['bbox'] = bbox_new
annotations_new[0]['area'] = (max_pixel_x-min(pixel_x))*(max_pixel_y-min(pixel_y))

In [79]:
data_new['annotations'] = annotations_new

In [80]:
print(data_new)

{'info': {'description': 'Pole Dataset', 'url': 'https://geranos.ethz.ch/', 'version': '1.0', 'year': 2022, 'contributor': 'Geranos ETHZ', 'date_created': '2022/04/27'}, 'licenses': [{'url': 'Unknown', 'id': 1, 'name': 'NONE'}], 'images': [{'license': 1, 'file_name': 'img1.png', 'height': 4032, 'width': 3024, 'date_captured': '2022-04-26 11:37:19', 'coco_url': 'Unknown', 'flickr_url': 'Unknown', 'id': 17}], 'annotations': [{'segmentation': [], 'num_keypoints': 8, 'area': 2348700.2200171966, 'iscrowd': 0, 'keypoints': [0.0, 0.0, 0, 1066.940234750033, 584.2850707014609, 2, 1769.2103204374218, 610.986377761447, 2, 0.0, 0.0, 0, 0.0, 0.0, 0, 1440.6376457977995, 3876.2608149411603, 2, 1697.9622605954507, 3928.7251485075167, 2, 0.0, 0.0, 0], 'image_id': 17, 'bbox': [1066.940234750033, 584.2850707014609, 702.2700856873889, 3344.440077806056], 'category_id': 420, 'id': 100000}], 'categories': [{'supercategory': 'poles', 'id': 420, 'name': 'pole', 'keypoints': ['star', 'rhomb', 'torp', 'tear', '

In [81]:
with open('coco_example_new.json', 'w') as outfile:
    json.dump(data_new, outfile)